In [14]:
import requests
import json
import pandas as pd
import time
import pprint
pp = pprint.PrettyPrinter(indent=4)


In [37]:
# Twitch Client ID 
clientID = 'vb2kmh60pt0tee6o2c11ko6n2t1w9a'

def get_top_100_games(clientID = clientID):
    ''' Given Client ID, pings twitch API for top 100 games. Returns the entire request object'''
    # Need to pass client ID with each request in header
    headers = {'Client-ID': clientID}
    url = '''https://api.twitch.tv/helix/games/top?first=100'''
    r = requests.get(url, headers=headers)
    return r

# r_dict = json.loads(r.text)

In [ ]:
r = get_top_100_games()
r_dict = json.loads(r.text)

In [9]:
stream_dict = get_top_100_streamers_for_each_game(r_dict)

Waiting for API limit to refresh (30s)...
Continuing...
Waiting for API limit to refresh (30s)...
Continuing...
Waiting for API limit to refresh (30s)...
Continuing...
Waiting for API limit to refresh (30s)...
Continuing...


In [75]:
stream_dict[0]

NameError: name 'stream_dict' is not defined

In [12]:
# Use this as the time stamp 
curr_time = time.strftime("%Y-%m-%d %H:%M:%S",time.localtime())

In [5]:
def check_api_limit_reached(req, ignore_limit = False):
    '''Check remaining API pings for request REQ. If API requests is <=1, wait for 30s 
    so for all requests to refill. Returns remaining requests'''
    if int(req.headers['Ratelimit-Remaining']) <= 1: # No more requests, need to pause for 30s
        if ignore_limit:
            return int(req.headers['Ratelimit-Remaining'])
        print('Waiting for API limit to refresh (30s)...')
        time.sleep(30)
        print('Continuing...')
    return int(req.headers['Ratelimit-Remaining'])

In [47]:
# Testing streams/game call
headers = {'Client-ID': clientID}
url = 'https://api.twitch.tv/helix/streams?first=10&game_id='
game_id_test = '32399'
req = requests.get(url + game_id_test,headers=headers)

In [48]:
json.loads(req.text)

{'data': [{'id': '34350754848',
   'user_id': '22859264',
   'user_name': 'DreamHackCS',
   'game_id': '32399',
   'community_ids': ['2f27ff6d-97fb-4df7-886d-c7893a13a13f',
    '4bef9cc9-c81d-4384-a62e-b16cf5e7a45e',
    '87d145b7-30a3-421d-a77f-4ef22dc51f90'],
   'type': 'live',
   'title': 'LIVE: Furia vs. Vitality - Quarter-Final #1 - CORSAIR DreamHack Masters Dallas 2019',
   'viewer_count': 22098,
   'started_at': '2019-05-31T18:07:43Z',
   'language': 'en',
   'thumbnail_url': 'https://static-cdn.jtvnw.net/previews-ttv/live_user_dreamhackcs-{width}x{height}.jpg',
   'tag_ids': ['6ea6bca4-4712-4ab9-a906-e3336a9d8039']},
  {'id': '34347844416',
   'user_id': '60917582',
   'user_name': 's1mple',
   'game_id': '32399',
   'community_ids': [],
   'type': 'live',
   'title': '@s1mpleO streaming! Fortnite + CS ',
   'viewer_count': 8400,
   'started_at': '2019-05-31T13:36:25Z',
   'language': 'en',
   'thumbnail_url': 'https://static-cdn.jtvnw.net/previews-ttv/live_user_s1mple-{width}x

In [49]:
def get_top_100_streamers_for_each_game(game_dict):
    '''Given the twitch response for top 100 games, this will cycle through and pull the top 100
    streamers for each game, stored under a dict entry of the title of that game'''
    stream_dict = dict()
    headers = {'Client-ID': clientID}
    url = 'https://api.twitch.tv/helix/streams?first=100&game_id='
    for game in game_dict['data']:
        req = requests.get(url + game['id'],headers=headers)
        check_api_limit_reached(req)    
        stream_dict[game['name']]=json.loads(req.text)
    return stream_dict

In [10]:
# stream_dict = dict()
# follower_dict = dict()
# streamer_url = 'https://api.twitch.tv/helix/streams?first=100&gameid=' # + game['id']
# follower_url = 'https://api.twitch.tv/helix/users/follows?first=1&to_id='# + game['userid']

# for game in r_dict['data']:
#     req = requests.get(streamer_url + game['id'],headers=headers)
#     check_api_limit_reached(req)    
    
#     # Get follower data
#     for stream in json.loads(req.text)['data']:
#         follow_req = requests.get(follower_url + stream['user_id'],headers=headers)
#         follower_dict[stream['user_id']] = json.loads(follow_req.text)['total']
#         check_api_limit_reached(follow_req)    

#     stream_dict[game['name']]=json.loads(req.text)
# ç

In [14]:
!ls

2019-05-30 21:19:55.json data_file.json
Scraper.ipynb            output.p


In [13]:
with open(curr_time + ".json", "w") as write_file:
    json.dump(stream_dict, write_file)

In [122]:
del stream_dict

In [3]:
file_name = '2019-05-30 21:19:55.json'
with open(file_name, "r") as read_file:
    data = json.load(read_file)

In [90]:
data['Fortnite']['data'][0]

{'id': '34340564320',
 'user_id': '22859264',
 'user_name': 'DreamHackCS',
 'game_id': '32399',
 'community_ids': ['2f27ff6d-97fb-4df7-886d-c7893a13a13f',
  '4bef9cc9-c81d-4384-a62e-b16cf5e7a45e',
  '87d145b7-30a3-421d-a77f-4ef22dc51f90'],
 'type': 'live',
 'title': 'LIVE: ENCE vs. Furia - BO3 - Group A - CORSAIR DreamHack Masters Dallas 2019',
 'viewer_count': 55056,
 'started_at': '2019-05-30T22:40:15Z',
 'language': 'en',
 'thumbnail_url': 'https://static-cdn.jtvnw.net/previews-ttv/live_user_dreamhackcs-{width}x{height}.jpg',
 'tag_ids': ['6ea6bca4-4712-4ab9-a906-e3336a9d8039']}

In [ ]:
try:
    # Stream data table
    # Grabs fields for insertion into dataframe
    user_id = stream['user_id']
    user_name = stream['user_name']
    game_id = stream['game_id']
    stream_type = stream['type']
    title = stream['title']
    viewer_count = stream['viewer_count']
    started_at = stream['started_at']
    language = ['language']
    
except:
    print('Failed to import a column')

In [15]:
def get_game_tags(clientID):
    # API call to for game tags
    return game_tags_json

In [34]:
lol_df = pd.io.json.json_normalize(data['League of Legends']['data'][:])

In [57]:
lol_df.shape

(100, 12)

In [82]:
type(data)

dict

In [58]:
fn_df.shape

(100, 12)

In [89]:
for game in data:
    #print(game)

SyntaxError: unexpected EOF while parsing (<ipython-input-89-8b05c3bd7130>, line 2)

In [99]:
type(data.keys())

dict_keys

In [101]:
for game_key in list(data.keys()):
    print(data[game_key])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [110]:
key_list = list()
pd.io.json.json_normalize(data[list(data.keys())[2]]['data'][90])
# for game_key in data.keys():
#     print(data[game_key])

,community_ids,game_id,id,language,started_at,tag_ids,thumbnail_url,title,type,user_id,user_name,viewer_count
0,"[399b1e72-643f-4f17-9ec5-1151b3a67578, 848d95b...",19745,34341217824,en,2019-05-30T23:33:58Z,"[ba2c968b-867a-49ce-aebc-3d978a204f4a, 6ea6bca...",https://static-cdn.jtvnw.net/previews-ttv/live...,Pokemon Platinum Nuzlocke Victory Road and bey...,live,30080840,TSM_TheOddOne,1687


In [123]:
total_streams_df = pd.DataFrame(
    columns = ['game_id','id','language','started_at','title','type','user_id','user_name','viewer_count'])
for game_key in list(data.keys()):
    game_streams_df = pd.io.json.json_normalize(data[game_key]['data'])
    total_streams_df = pd.concat([total_streams_df, game_streams_df], sort = False)
total_streams_df.drop(columns = ['community_ids','thumbnail_url','tag_ids'], inplace = True)

In [124]:
len(total_streams_df.columns)

9

In [125]:
total_streams_df.shape

(9900, 9)

In [70]:
lol_df=pd.concat([lol_df,fn_df])
lol_df.shape

(500, 12)

In [74]:
lol_df.iloc[100]

community_ids    [2f27ff6d-97fb-4df7-886d-c7893a13a13f, 4bef9cc...
game_id                                                      32399
id                                                     34340564320
language                                                        en
started_at                                    2019-05-30T22:40:15Z
tag_ids                     [6ea6bca4-4712-4ab9-a906-e3336a9d8039]
thumbnail_url    https://static-cdn.jtvnw.net/previews-ttv/live...
title            LIVE: ENCE vs. Furia - BO3 - Group A - CORSAIR...
type                                                          live
user_id                                                   22859264
user_name                                              DreamHackCS
viewer_count                                                 55056
Name: 0, dtype: object

In [62]:
pd.DataFrame([lol_df,fn_df], 
             columns = ['game_id','id','language','started_at','title','type','user_id','user_name','viewer_count'])

ValueError: Shape of passed values is (2, 1), indices imply (2, 9)

In [55]:
fn_df = pd.io.json.json_normalize(data['Fortnite']['data'][:])

In [31]:
fn_df.drop(columns = ['community_ids','thumbnail_url','tag_ids'])

,game_id,id,language,started_at,title,type,user_id,user_name,viewer_count
0,32399,34340564320,en,2019-05-30T22:40:15Z,LIVE: ENCE vs. Furia - BO3 - Group A - CORSAIR...,live,22859264,DreamHackCS,55056
1,509577,34342686896,en,2019-05-31T01:39:57Z,"Critical Role Campaign 2, Episode 65 (5/30 at ...",live,229729353,CriticalRole,49239
2,33214,34343599184,en,2019-05-31T03:10:11Z,Code: Tsm_hamlinz,live,67143805,TSM_Hamlinz,20143
3,491704,34339443648,en,2019-05-30T21:06:12Z,M1 BUFF!! | @shroud on socials for updates,live,37402112,shroud,18950
4,32982,34341977600,en,2019-05-31T00:37:17Z,Mr.Chang Detective | Nopixel RP | !Twitter @Lo...,live,163836275,Lord_Kebun,17980
5,33214,34338086864,en,2019-05-30T19:22:14Z,Duos With Brooke | Use Code Sym,live,31688366,Symfuhny,16427
6,460508,34338594688,en,2019-05-30T20:00:46Z,No zombie is safe,live,71092938,xQcOW,14626
7,33214,34337782704,en,2019-05-30T19:00:18Z,JOINED 100 THIEVES! NEW PODCAST !podcast MERCH...,live,106125347,CouRageJD,14584
8,18122,34341324352,en,2019-05-30T23:43:13Z,chill leveling [ @summit1g ],live,26490481,summit1g,14126
9,510268,34342918272,en,2019-05-31T02:01:34Z,!gamejam | NEW GAME conquered YET AGAIN by COS...,live,121059319,MOONMOON_OW,13633


In [26]:
CREATE TABLE stream_data (
 stream_id int,
 user_id int,
 user_name text,
 game_id int,
 stream_type text,
 title text,
 viewer_count int,
 started_at timestamp,
 language text);

 CREATE TABLE game_data (
 game_id int,
 game_name text);

CREATE TABLE game_tags (
tag_id int,
tag_english


SyntaxError: invalid syntax (<ipython-input-26-64529465d538>, line 1)

In [158]:
whos

Variable                              Type             Data/Info
----------------------------------------------------------------
check_api_limit                       function         <function check_api_limit at 0x11eb45b70>
check_api_limit_reached               function         <function check_api_limit_reached at 0x126d38950>
clientID                              str              vb2kmh60pt0tee6o2c11ko6n2t1w9a
create_engine                         function         <function create_engine at 0x11b2c68c8>
data                                  dict             n=99
game                                  dict             n=3
get_top_100_games                     function         <function get_top_100_games at 0x126d277b8>
get_top_100_streamers_for_each_game   function         <function get_top_100_str<...>each_game at 0x126d279d8>
headers                               dict             n=1
json                                  module           <module 'json' from '/ana<...>hon3.7/json/__

In [28]:
!pwd

/Users/jchow/Work/MetisCode/twitch-streamer-recommender/gitRoot/Twitch_Streamers_Recommender/Scraper_Notebook


In [34]:
tfue_id = output_list[0]['data'][0]['user_id']
#[0]['user_id']

'{"total":6166637,"data":[{"from_id":"438578472","from_name":"hentayuu","to_id":"60056333","to_name":"Tfue","followed_at":"2019-05-31T02:15:32Z"}],"pagination":{"cursor":"eyJiIjpudWxsLCJhIjp7IkN1cnNvciI6IjE1NTkyNjg5MzIyMzk5MDQzNTQifX0"}}'

In [153]:
json.loads(testrequest)['total']

6166637

In [90]:
url = 'https://api.twitch.tv/helix/users/follows?first=1&to_id=' + tfue_id
testrequest=requests.get(url,headers=headers).text

In [39]:
# Example api ping through command line
# !curl -H 'Client-ID: vb2kmh60pt0tee6o2c11ko6n2t1w9a' \-X GET https://api.twitch.tv/helix/games/top